In [1]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: 

# RAG버전

In [15]:
# =========================
# 설치 (필요시 1회)
# =========================
!pip -q install -U transformers accelerate peft trl datasets pandas bitsandbytes rich
!pip -q install langchain langchain-community faiss-cpu sentence-transformers

# =========================
# 0) 공용 import & Drive 마운트
# =========================
import os, re, json, pandas as pd, torch
from typing import List, Optional, Tuple
from collections import Counter
from google.colab import drive
drive.mount('/content/drive')

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# LangChain FAISS/E5
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# =========================
# 1) 고정 경로/파라미터
# =========================
MAX_SEQ_LENGTH = 20000
SHUFFLE_SEED   = 42
TEST_LIMIT     = 3
KEYS           = ["subject_id", "hadm_id", "note_id"]

MODEL_DIR   = "/content/drive/MyDrive/DILAB/Qwen2-7B-Instruct"
ADAPTER_DIR = "/content/drive/MyDrive/DILAB/Qwen2-7B-Instruct/adapter/original_1.3"
BASE_DATA   = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/files/note/input_output"

TEST_INPUT  = f"{BASE_DATA}/discharge_test_input.parquet"
TEST_OUTPUT = f"{BASE_DATA}/discharge_test_output.parquet"

# ==== (RAG) FAISS/E5 경로 ====
FAISS_DIR  = "/content/drive/MyDrive/DILAB/MARS/UMLS/faiss_e5_base"
E5_MODEL   = "intfloat/e5-base-v2"   # 한글 질의 섞이면 "intfloat/multilingual-e5-base"

# =========================
# 2) Qwen2 + 어댑터 로드
# =========================
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_DIR,
    quantization_config=bnb_config,
    dtype=torch.bfloat16,
    max_length=MAX_SEQ_LENGTH,
    device_map={"": 0},
    offload_buffers=True
)
trained_model = PeftModel.from_pretrained(
    base_model,
    ADAPTER_DIR,
    is_trainable=False
)
trained_model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.max_seq_length = MAX_SEQ_LENGTH

print("✅ Trained model & tokenizer loaded")

# =========================
# 3) 데이터 로드 & 샘플 선택
# =========================
test_input_df  = pd.read_parquet(TEST_INPUT)
test_output_df = pd.read_parquet(TEST_OUTPUT)
print(f"✅ TEST INPUT rows: {len(test_input_df):,}")
print(f"✅ TEST OUTPUT rows: {len(test_output_df):,}")

test_merged_df = test_input_df.merge(test_output_df, on=KEYS, how="inner")
both_empty = (test_merged_df["bhc_text"].fillna("").str.len() == 0) & \
             (test_merged_df["di_text"].fillna("").str.len() == 0)
test_merged_df = test_merged_df[~both_empty].reset_index(drop=True)

if TEST_LIMIT is not None and TEST_LIMIT < len(test_merged_df):
    sample_df = test_merged_df.sample(n=TEST_LIMIT, random_state=SHUFFLE_SEED).reset_index(drop=True)
else:
    sample_df = test_merged_df.copy()

print(f"🎯 최종 선택된 테스트 샘플 수: {len(sample_df):,}")

# =========================
# 4) 지시문(역할 설명)
# =========================
# - BHC는 원문만 사용 (RAG 금지) + 문제 리스트 '#' 금지 명시
BHC_INSTR = """
You are an expert physician-writer. Write ONLY the “Brief Hospital Course (BHC)”
for THIS admission using facts explicitly present in the INPUT note.

HARD RULES
- Cite ONLY facts that appear verbatim in the input; do NOT invent meds, doses, procedures, dates.
- Prefer ≤8 bullets or a compact paragraph.
- Chronologic clarity: presentation → key findings → diagnostics → treatments/changes with reasons → response/status at discharge → pending items.
- Keep it clinical, crisp, and de-duplicated.
- DO NOT output any problem-list lines starting with '#'. If a problem list exists in the source, rewrite concisely as bullets ('-') or inline text.

OUTPUT
Brief Hospital Course
<content only; DO NOT output any other headings>
"""

# - DI는 원문 지시문 + (별도로) RAG evidence 힌트만 덧붙임
DI_INSTR = """
You are an expert physician-writer. Produce “Discharge Instructions (DI)”
STRICTLY from patient-facing instructions present in the INPUT note.

HARD RULES
- Include ONLY explicit instructions (medication directions/changes with dose/frequency if present;
  activity/wound care; when/where to follow up; monitoring; diet).
- If there are NO explicit instructions, output exactly one line:
  No explicit discharge instructions found in source.

STYLE
- Bulleted, concise, patient-facing language.
- DO NOT invent advice.

OUTPUT
Discharge Instructions
<patient-facing bullets only; DO NOT add other sections>
"""

# =========================
# 5) (RAG) FAISS + E5 임베딩 로드 & 리트리버 (DI 전용)
# =========================
emb = HuggingFaceEmbeddings(
    model_name=E5_MODEL,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True, "batch_size": 64},
)
vs = FAISS.load_local(FAISS_DIR, emb, allow_dangerous_deserialization=True)

retriever = vs.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 8, "fetch_k": 50, "lambda_mult": 0.5}
)

def _strip_passage_prefix(text: str) -> str:
    return text.replace("passage: ", "", 1) if text.startswith("passage: ") else text

# ---- 노이즈 판정(증거/출력 공통) ----
_noise_hash_re = re.compile(r"(?:^|\s)#[A-Za-z][A-Za-z0-9\- ]{2,}")

def is_noise_block(text: str) -> bool:
    t = text.strip()
    if not t:
        return True
    # 해시태그 과다
    if len(_noise_hash_re.findall(t)) >= 5 or t.count("#") >= 8:
        return True
    # 단어 과도 + 문장부호 적음 => 사전/나열류
    words = re.findall(r"\b[A-Za-z]{3,}\b", t)
    if len(words) > 250 and t.count(".") < 5:
        return True
    # 단일 토큰 과반 반복
    if words:
        c = Counter([w.lower() for w in words])
        topn = c.most_common(1)[0][1]
        if topn / max(1, len(words)) > 0.2:
            return True
    return False

def compress_block(b: str, max_chars=420) -> str:
    b = _strip_passage_prefix(b.strip())
    def pick(label):
        m = re.search(fr'{label}:\s*(.+)', b, re.I)
        return (label, m.group(1).strip()) if m else None
    keys = ["disease name", "disease description", "drug name", "relationship attribute"]
    lines = []
    for k in keys:
        p = pick(k)
        if p: lines.append(f"{p[0]}: {p[1]}")
    s = "\n".join(lines) if lines else b
    return s[:max_chars] + (" ..." if len(s) > max_chars else "")

def format_evidence_blocks(blocks: List[str]) -> str:
    lines = []
    for i, blk in enumerate(blocks[:8], 1):  # 최대 8개
        lines.append(f"### EVIDENCE #{i}\n{compress_block(blk)}")
    return "\n\n".join(lines)

HEADER_PATTERNS = [
    r"(?im)^\s*Discharge Diagnosis\s*:\s*(.+)$",
    r"(?im)^\s*Discharge Dx\s*:\s*(.+)$",
    r"(?im)^\s*Diagnosis\s*:\s*(.+)$",
    r"(?im)^\s*Chief Complaint\s*:\s*(.+)$",
    r"(?im)^\s*CC\s*:\s*(.+)$",
]
SECTION_PATTERNS = [
    r"(?is)assessment\s*[:\-]\s*(.+?)(?:\n[A-Z][^\n]{0,40}[:\-]|$)",
    r"(?is)plan\s*[:\-]\s*(.+?)(?:\n[A-Z][^\n]{0,40}[:\-]|$)",
    r"(?is)principal\s+diagnosis\s*[:\-]\s*(.+)",
    r"(?is)discharge\s+diagnosis\s*[:\-]\s*(.+)",
]

def make_queries_from_note(note: str, extra_hints: Optional[List[str]] = None) -> List[str]:
    qs = []
    for pat in HEADER_PATTERNS:
        for m in re.finditer(pat, note):
            line = m.group(1).strip()
            if line:
                qs.append(f"query: {line}")
    key_terms = []
    for pat in SECTION_PATTERNS:
        m = re.search(pat, note)
        if m:
            chunk = re.sub(r"\s+", " ", m.group(1))[:300]
            for w in re.split(r"[,/;]| and | with ", chunk)[:6]:
                w = w.strip()
                if len(w) >= 4:
                    key_terms.append(w)
    for w in key_terms[:6]:
        qs.append(f"query: {w}")
    snippet = re.sub(r"\s+", " ", note.strip())[:500]
    if snippet:
        qs.append(f"query: {snippet}")
    if extra_hints:
        for h in extra_hints:
            qs.append(f"query: {h}")
    seen = set(); uniq=[]
    for q in qs:
        if q not in seen:
            seen.add(q); uniq.append(q)
    return uniq[:6]

def retrieve_evidence(note_text: str) -> List[str]:
    queries = make_queries_from_note(note_text)
    docs = []
    for q in queries:
        docs.extend(retriever.get_relevant_documents(q))
    seen = set(); uniq = []
    for d in docs:
        t = _strip_passage_prefix(d.page_content.strip())
        if t in seen or is_noise_block(t):
            continue
        seen.add(t)
        uniq.append(t)
        if len(uniq) >= 12:
            break
    return uniq

def parse_evidence_pairs(blocks: List[str]) -> List[Tuple[str, str]]:
    pairs = []
    for b in blocks:
        if is_noise_block(b):
            continue
        dis = re.search(r"(?im)^disease name:\s*(.+)$", b)
        drg = re.search(r"(?im)^drug name:\s*(.+)$", b)
        if dis and drg:
            disease = re.split(r"[\(\[,;/]", dis.group(1).strip())[0].strip()
            drug    = re.split(r"[\(\[,;/]", drg.group(1).strip())[0].strip()
            if len(disease) >= 3 and len(drug) >= 2:
                pairs.append((disease, drug))
    return pairs

# =========================
# 6) 생성 유틸 + 노이즈 클린
# =========================
def _serialize_chat(messages: List[dict]) -> str:
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

@torch.inference_mode()
def _generate_text(model, prompt: str, max_new_tokens: int = 800) -> str:
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    gen_ids = model.generate(
        inputs.input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        repetition_penalty=1.20,        # ⬆️ 반복 억제 강화
        no_repeat_ngram_size=7,         # ⬆️ 구문 반복 억제 강화
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )
    gen_only = [out[len(inp):] for inp, out in zip(inputs.input_ids, gen_ids)]
    return tokenizer.batch_decode(gen_only, skip_special_tokens=True)[0].strip()

def build_bhc_prompt(note_text: str) -> str:
    messages = [
        {"role": "system", "content": BHC_INSTR},
        {"role": "user",   "content": note_text.strip()},
        {"role": "assistant", "content": ""}
    ]
    return _serialize_chat(messages)

def build_di_prompt(note_text: str) -> str:
    messages = [
        {"role": "system", "content": DI_INSTR},
        {"role": "user",   "content": note_text.strip()},
        {"role": "assistant", "content": ""}
    ]
    return _serialize_chat(messages)

# ---- 원문과 연속 일치 검사(폭주 라인 거르기) ----
def has_long_overlap(line: str, source: str, min_run: int = 8) -> bool:
    line_norm = re.sub(r'\s+', ' ', line).strip().lower()
    src_norm  = re.sub(r'\s+', ' ', source).strip().lower()
    if len(line_norm) < min_run:
        return False
    for i in range(0, len(line_norm) - min_run + 1):
        if line_norm[i:i+min_run] in src_norm:
            return True
    return False

# ---- 생성 결과 노이즈 클리너 ----
def clean_model_output(
    text: str,
    source_note: str,
    allow_hash: bool = False,
    max_hash_lines: int = 6,
    runaway_threshold: int = 5,
) -> str:
    lines = text.splitlines()
    cleaned = []
    hash_count = 0
    runaway_hash_seq = 0
    for ln in lines:
        raw = ln.rstrip()

        # 완전 공백 줄 정리(연속 두 줄 이상 방지)
        if not raw.strip():
            if cleaned and cleaned[-1] == "":
                continue
            cleaned.append("")
            continue

        # 해시태그/사전류 폭주 라인
        if raw.lstrip().startswith("#"):
            # 허용하지 않거나, 원문과 겹치지 않으면 스킵
            if (not allow_hash) or (not has_long_overlap(raw, source_note, 8)):
                runaway_hash_seq += 1
                if runaway_hash_seq >= runaway_threshold:
                    # 여기서 아예 절단: 이후 라인은 버림
                    break
                continue
            # 허용된 '#'(원문 겹침)이라도 총량 제한
            if hash_count >= max_hash_lines:
                break
            hash_count += 1
            runaway_hash_seq = 0
            cleaned.append(raw)
            continue

        # 사전형 나열(쉼표/세미콜론 과도, 대문자 단어 나열 등) — 원문과 겹침 없으면 컷
        token_words = re.findall(r"[A-Za-z]{3,}", raw)
        if (len(token_words) > 40 and raw.count(",") + raw.count(";") >= 6) and not has_long_overlap(raw, source_note, 8):
            continue

        # 동일 라인 반복 방지
        if cleaned and cleaned[-1] == raw:
            continue

        runaway_hash_seq = 0
        cleaned.append(raw)

    # 머리/꼬리 공백 정리 + 빈 줄 2개 제한
    out = "\n".join(cleaned)
    out = re.sub(r"\n{3,}", "\n\n", out).strip()
    return out

def build_evidence_hints(note_text: str) -> List[str]:
    blocks = retrieve_evidence(note_text)
    pairs  = parse_evidence_pairs(blocks)
    note_lc = note_text.lower()

    hints = []
    seen  = set()
    for dis, drg in pairs:
        if dis.lower() in note_lc and (dis, drg) not in seen:
            if 2 <= len(drg) <= 50 and 3 <= len(dis) <= 80:
                hints.append(f"- For {dis}: discuss with your clinician whether **{drg}** is appropriate.")
                seen.add((dis, drg))
        if len(hints) >= 6:
            break
    return hints

def assemble_final_output(bhc_text: str, di_from_note: str, ev_hints: List[str]) -> str:
    out = []

    # BHC
    if not re.search(r'(?im)^Brief Hospital Course\s*$', bhc_text):
        out.append("Brief Hospital Course")
    out.append(bhc_text.strip())

    # DI (원문 지시문)
    if not re.search(r'(?im)^Discharge Instructions\s*$', di_from_note):
        out.append("\nDischarge Instructions")
    out.append(di_from_note.strip())

    # DI (Evidence Hints)
    if ev_hints:
        out.append("\nEvidence Hints (from disease↔drug knowledge)")
        out.append("\n".join(ev_hints))

    return "\n".join(out).strip()

# =========================
# (추가) 정답(GT) 리스트
# =========================
outputs = sample_df[["bhc_text", "di_text"]].fillna("").to_dict("records")
print(f"✅ Ground-truth outputs attached for {len(outputs)} samples.")

# =========================
# 7) 실행: BHC(원문), DI(원문) + Evidence Hints(벡터DB)
# =========================
N = min(3, len(sample_df))
for i in range(N):
    note_text = sample_df.loc[i, "text"]
    print(f"\n\n------------- CASE {i+1}/{N} ---------------")

    # 1) BHC: 원문만 사용 (RAG 미사용)
    bhc_prompt = build_bhc_prompt(note_text)
    bhc_raw    = _generate_text(trained_model, bhc_prompt, max_new_tokens=800)
    # ⬇️ 노이즈 클린: BHC는 '#' 미허용
    bhc_text   = clean_model_output(bhc_raw, note_text, allow_hash=False)

    # 2) DI: 원문만으로 LLM 생성
    di_prompt  = build_di_prompt(note_text)
    di_raw     = _generate_text(trained_model, di_prompt, max_new_tokens=600)
    # ⬇️ 노이즈 클린: DI에서도 폭주 패턴 제거
    di_text    = clean_model_output(di_raw, note_text, allow_hash=False)

    # 3) Evidence Hints: 벡터DB 검색 → (질병→약물) 힌트
    ev_hints   = build_evidence_hints(note_text)

    # 4) 합치기
    final_text = assemble_final_output(bhc_text, di_text, ev_hints)
    print(final_text)

    # 5) GT(정답)도 함께 출력
    print(f"\n\n-------------------output-------------------------------")
    print("BHC (GT)")
    print(outputs[i]["bhc_text"] if outputs[i]["bhc_text"] else "(empty)")
    print("DI (GT)")
    print(outputs[i]["di_text"] if outputs[i]["di_text"] else "No explicit discharge instructions")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Trained model & tokenizer loaded
✅ TEST INPUT rows: 68,589
✅ TEST OUTPUT rows: 68,589
🎯 최종 선택된 테스트 샘플 수: 3
✅ Ground-truth outputs attached for 3 samples.


------------- CASE 1/3 ---------------
Brief Hospital Course
The patient underwent laparotomy on ___ which revealed a large
pancreatic mass extending into the head of the pancreas. The
patient tolerated the procedure well without complications. She
was extubated post-operatively and transferred to PACU where
she remained overnight before being admitted to the floor. On
the morning following surgery, the patient reported feeling
nauseous but not vomiting. Her nausea improved over time until
it resolved completely. She also complained of abdominal pain
which responded to IV PCA. She was weaned off PCA after two days
and transitioned onto oral analgesics. She did require some
IVF during this period due to dehydration from poor appetite.
Her diet was advanced slowly beginning on POD+2 when she could
tolerate regular food. She was disc

In [14]:
# =========================
# 설치 (필요시 1회)
# =========================
!pip -q install -U transformers accelerate peft trl datasets pandas bitsandbytes rich
!pip -q install langchain langchain-community faiss-cpu sentence-transformers

# =========================
# 0) 공용 import & Drive 마운트
# =========================
import os, re, json, pandas as pd, torch
from typing import List, Optional, Tuple
from collections import Counter
from google.colab import drive
drive.mount('/content/drive')

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# LangChain FAISS/E5
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# =========================
# 1) 고정 경로/파라미터
# =========================
MAX_SEQ_LENGTH = 20000
SHUFFLE_SEED   = 42
TEST_LIMIT     = 3
KEYS           = ["subject_id", "hadm_id", "note_id"]

MODEL_DIR   = "/content/drive/MyDrive/DILAB/Qwen2-7B-Instruct"
ADAPTER_DIR = "/content/drive/MyDrive/DILAB/Qwen2-7B-Instruct/adapter/original_1.3"
BASE_DATA   = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/files/note/input_output"

TEST_INPUT  = f"{BASE_DATA}/discharge_test_input.parquet"
TEST_OUTPUT = f"{BASE_DATA}/discharge_test_output.parquet"

# ==== (RAG) FAISS/E5 경로 ====
FAISS_DIR  = "/content/drive/MyDrive/DILAB/MARS/UMLS/faiss_e5_base"
E5_MODEL   = "intfloat/e5-base-v2"   # 한글 질의 섞이면 "intfloat/multilingual-e5-base"

# =========================
# 2) Qwen2 + 어댑터 로드
# =========================
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_DIR,
    quantization_config=bnb_config,
    dtype=torch.bfloat16,
    max_length=MAX_SEQ_LENGTH,
    device_map={"": 0},
    offload_buffers=True
)
trained_model = PeftModel.from_pretrained(
    base_model,
    ADAPTER_DIR,
    is_trainable=False
)
trained_model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.max_seq_length = MAX_SEQ_LENGTH

print("✅ Trained model & tokenizer loaded")

# =========================
# 3) 데이터 로드 & 샘플 선택
# =========================
test_input_df  = pd.read_parquet(TEST_INPUT)
test_output_df = pd.read_parquet(TEST_OUTPUT)
print(f"✅ TEST INPUT rows: {len(test_input_df):,}")
print(f"✅ TEST OUTPUT rows: {len(test_output_df):,}")

test_merged_df = test_input_df.merge(test_output_df, on=KEYS, how="inner")
both_empty = (test_merged_df["bhc_text"].fillna("").str.len() == 0) & \
             (test_merged_df["di_text"].fillna("").str.len() == 0)
test_merged_df = test_merged_df[~both_empty].reset_index(drop=True)

if TEST_LIMIT is not None and TEST_LIMIT < len(test_merged_df):
    sample_df = test_merged_df.sample(n=TEST_LIMIT, random_state=SHUFFLE_SEED).reset_index(drop=True)
else:
    sample_df = test_merged_df.copy()

print(f"🎯 최종 선택된 테스트 샘플 수: {len(sample_df):,}")

# =========================
# 4) 지시문(역할 설명)
# =========================
# - BHC는 원문만 사용 (RAG 금지) + 문제 리스트 '#' 금지 명시
BHC_INSTR = """
You are an expert physician-writer. Write ONLY the “Brief Hospital Course (BHC)”
for THIS admission using facts explicitly present in the INPUT note.

HARD RULES
- Cite ONLY facts that appear verbatim in the input; do NOT invent meds, doses, procedures, dates.
- Prefer ≤8 bullets or a compact paragraph.
- Chronologic clarity: presentation → key findings → diagnostics → treatments/changes with reasons → response/status at discharge → pending items.
- Keep it clinical, crisp, and de-duplicated.
- DO NOT output any problem-list lines starting with '#'. If a problem list exists in the source, rewrite concisely as bullets ('-') or inline text.

OUTPUT
Brief Hospital Course
<content only; DO NOT output any other headings>
"""

# - DI는 원문 지시문 + (별도로) RAG evidence 힌트만 덧붙임
DI_INSTR = """
You are an expert physician-writer. Produce “Discharge Instructions (DI)”.

EVIDENCE RULES
- You will be given "EVIDENCE BLOCKS" from a knowledge base.
- Use these evidence blocks STRICTLY to inform potential medication advice related to diagnoses.
- If an evidence block includes contraindications or 'contraindicated' terms,
  reflect this as a caution rather than recommending the drug.
- DO NOT mention diseases or drugs from the evidence unless they are also
  supported by the main "INPUT NOTE".

INPUT NOTE RULES
- Also use the "INPUT NOTE" to find patient-facing instructions.
- Include ONLY explicit instructions (medication directions/changes with dose/frequency if present;
  activity/wound care; when/where to follow up; monitoring; diet).
- If there are NO explicit instructions in the INPUT NOTE and NO relevant evidence, output exactly one line:
  No explicit discharge instructions found in source.

STYLE
- Bulleted, concise, patient-facing language.

OUTPUT
Discharge Instructions
<patient-facing bullets only; DO NOT add other sections>
"""

# =========================
# 5) (RAG) FAISS + E5 임베딩 로드 & 리트리버 (DI 전용)
# =========================
emb = HuggingFaceEmbeddings(
    model_name=E5_MODEL,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True, "batch_size": 64},
)
vs = FAISS.load_local(FAISS_DIR, emb, allow_dangerous_deserialization=True)

retriever = vs.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 8, "fetch_k": 50, "lambda_mult": 0.5}
)

def _strip_passage_prefix(text: str) -> str:
    return text.replace("passage: ", "", 1) if text.startswith("passage: ") else text

# ---- 노이즈 판정(증거/출력 공통) ----
_noise_hash_re = re.compile(r"(?:^|\s)#[A-Za-z][A-Za-z0-9\- ]{2,}")

def is_noise_block(text: str) -> bool:
    t = text.strip()
    if not t:
        return True
    # 해시태그 과다
    if len(_noise_hash_re.findall(t)) >= 5 or t.count("#") >= 8:
        return True
    # 단어 과도 + 문장부호 적음 => 사전/나열류
    words = re.findall(r"\b[A-Za-z]{3,}\b", t)
    if len(words) > 250 and t.count(".") < 5:
        return True
    # 단일 토큰 과반 반복
    if words:
        c = Counter([w.lower() for w in words])
        topn = c.most_common(1)[0][1]
        if topn / max(1, len(words)) > 0.2:
            return True
    return False

def compress_block(b: str, max_chars=420) -> str:
    b = _strip_passage_prefix(b.strip())
    def pick(label):
        m = re.search(fr'{label}:\s*(.+)', b, re.I)
        return (label, m.group(1).strip()) if m else None
    keys = ["disease name", "disease description", "drug name", "relationship attribute"]
    lines = []
    for k in keys:
        p = pick(k)
        if p: lines.append(f"{p[0]}: {p[1]}")
    s = "\n".join(lines) if lines else b
    return s[:max_chars] + (" ..." if len(s) > max_chars else "")

def format_evidence_blocks(blocks: List[str]) -> str:
    lines = []
    for i, blk in enumerate(blocks[:8], 1):  # 최대 8개
        lines.append(f"### EVIDENCE #{i}\n{compress_block(blk)}")
    return "\n\n".join(lines)

HEADER_PATTERNS = [
    r"(?im)^\s*Discharge Diagnosis\s*:\s*(.+)$",
    r"(?im)^\s*Discharge Dx\s*:\s*(.+)$",
    r"(?im)^\s*Diagnosis\s*:\s*(.+)$",
    r"(?im)^\s*Chief Complaint\s*:\s*(.+)$",
    r"(?im)^\s*CC\s*:\s*(.+)$",
]
SECTION_PATTERNS = [
    r"(?is)assessment\s*[:\-]\s*(.+?)(?:\n[A-Z][^\n]{0,40}[:\-]|$)",
    r"(?is)plan\s*[:\-]\s*(.+?)(?:\n[A-Z][^\n]{0,40}[:\-]|$)",
    r"(?is)principal\s+diagnosis\s*[:\-]\s*(.+)",
    r"(?is)discharge\s+diagnosis\s*[:\-]\s*(.+)",
]

def make_queries_from_note(note: str, extra_hints: Optional[List[str]] = None) -> List[str]:
    qs = []
    for pat in HEADER_PATTERNS:
        for m in re.finditer(pat, note):
            line = m.group(1).strip()
            if line:
                qs.append(f"query: {line}")
    key_terms = []
    for pat in SECTION_PATTERNS:
        m = re.search(pat, note)
        if m:
            chunk = re.sub(r"\s+", " ", m.group(1))[:300]
            for w in re.split(r"[,/;]| and | with ", chunk)[:6]:
                w = w.strip()
                if len(w) >= 4:
                    key_terms.append(w)
    for w in key_terms[:6]:
        qs.append(f"query: {w}")
    snippet = re.sub(r"\s+", " ", note.strip())[:500]
    if snippet:
        qs.append(f"query: {snippet}")
    if extra_hints:
        for h in extra_hints:
            qs.append(f"query: {h}")
    seen = set(); uniq=[]
    for q in qs:
        if q not in seen:
            seen.add(q); uniq.append(q)
    return uniq[:6]

def retrieve_evidence(note_text: str) -> List[str]:
    queries = make_queries_from_note(note_text)
    docs = []
    for q in queries:
        docs.extend(retriever.get_relevant_documents(q))
    seen = set(); uniq = []
    for d in docs:
        t = _strip_passage_prefix(d.page_content.strip())
        if t in seen or is_noise_block(t):
            continue
        seen.add(t)
        uniq.append(t)
        if len(uniq) >= 12:
            break
    return uniq

def parse_evidence_pairs(blocks: List[str]) -> List[Tuple[str, str]]:
    pairs = []
    for b in blocks:
        if is_noise_block(b):
            continue
        dis = re.search(r"(?im)^disease name:\s*(.+)$", b)
        drg = re.search(r"(?im)^drug name:\s*(.+)$", b)
        if dis and drg:
            disease = re.split(r"[\(\[,;/]", dis.group(1).strip())[0].strip()
            drug    = re.split(r"[\(\[,;/]", drg.group(1).strip())[0].strip()
            if len(disease) >= 3 and len(drug) >= 2:
                pairs.append((disease, drug))
    return pairs

# =========================
# 6) 생성 유틸 + 노이즈 클린
# =========================
def _serialize_chat(messages: List[dict]) -> str:
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

@torch.inference_mode()
def _generate_text(model, prompt: str, max_new_tokens: int = 800) -> str:
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    gen_ids = model.generate(
        inputs.input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        repetition_penalty=1.20,       # ⬆️ 반복 억제 강화
        no_repeat_ngram_size=7,        # ⬆️ 구문 반복 억제 강화
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )
    gen_only = [out[len(inp):] for inp, out in zip(inputs.input_ids, gen_ids)]
    return tokenizer.batch_decode(gen_only, skip_special_tokens=True)[0].strip()

def build_bhc_prompt(note_text: str) -> str:
    messages = [
        {"role": "system", "content": BHC_INSTR},
        {"role": "user",   "content": f"INPUT NOTE:\n{note_text.strip()}"},
        {"role": "assistant", "content": ""}
    ]
    return _serialize_chat(messages)

# ⭐ [수정] RAG 증거(evidence_text)를 프롬프트의 일부로 받도록 변경
def build_di_prompt(note_text: str, evidence_text: str) -> str:
    user_content = ""
    if evidence_text:
        user_content += f"EVIDENCE BLOCKS:\n{evidence_text}\n\n"
    user_content += f"INPUT NOTE:\n{note_text.strip()}"

    messages = [
        {"role": "system", "content": DI_INSTR},
        {"role": "user",   "content": user_content},
        {"role": "assistant", "content": ""}
    ]
    return _serialize_chat(messages)

# ---- (이하 clean_model_output 등 나머지 함수는 그대로 유지) ----
# ... (clean_model_output, has_long_overlap 등 함수들) ...
# ... (build_evidence_hints 대신 format_evidence_blocks를 직접 사용) ...

# ⭐ [수정] 최종 조립 함수 단순화 (이제 BHC와 DI만 합치면 됨)
def assemble_final_output(bhc_text: str, di_text: str) -> str:
    out = []

    # BHC
    if not re.search(r'(?im)^Brief Hospital Course\s*$', bhc_text):
        out.append("Brief Hospital Course")
    out.append(bhc_text.strip())

    # DI
    if not re.search(r'(?im)^Discharge Instructions\s*$', di_text):
        out.append("\nDischarge Instructions")
    out.append(di_text.strip())

    return "\n".join(out).strip()


# =========================
# (추가) 정답(GT) 리스트
# =========================
outputs = sample_df[["bhc_text", "di_text"]].fillna("").to_dict("records")
print(f"✅ Ground-truth outputs attached for {len(outputs)} samples.")

# =========================
# 7) 실행: BHC(원문), DI(RAG)
# =========================
N = min(3, len(sample_df))
for i in range(N):
    note_text = sample_df.loc[i, "text"]
    print(f"\n\n------------- CASE {i+1}/{N} ---------------")

    # 1) BHC: 원문만 사용 (RAG 미사용)
    bhc_prompt = build_bhc_prompt(note_text)
    bhc_raw    = _generate_text(trained_model, bhc_prompt, max_new_tokens=800)
    bhc_text   = clean_model_output(bhc_raw, note_text, allow_hash=False)

    # ⭐ [수정] 2) RAG 증거 먼저 검색
    # (build_evidence_hints 대신 retrieve_evidence 와 format_evidence_blocks 사용)
    retrieved_blocks = retrieve_evidence(note_text)
    evidence_text = format_evidence_blocks(retrieved_blocks)

    # RAG 검색 결과가 있을 경우에만 터미널에 출력 (디버깅용)
    if evidence_text:
        print(f"\n[Evidence fetched: {len(retrieved_blocks)} blocks]\n{evidence_text}")
    else:
        print("\n[No relevant evidence fetched]")

    # ⭐ [수정] 3) DI: 원문 + RAG 증거(evidence_text) 함께 주입
    di_prompt  = build_di_prompt(note_text, evidence_text)
    di_raw     = _generate_text(trained_model, di_prompt, max_new_tokens=600)
    di_text    = clean_model_output(di_raw, note_text, allow_hash=False)

    # ⭐ [수정] 4) 합치기 (이제 ev_hints 인자 불필요)
    final_text = assemble_final_output(bhc_text, di_text)
    print("\n\n-------------------FINAL MODEL OUTPUT-------------------")
    print(final_text)

    # 5) GT(정답)도 함께 출력
    print(f"\n\n-------------------output (GT)------------------------")
    print("BHC (GT)")
    print(outputs[i]["bhc_text"] if outputs[i]["bhc_text"] else "(empty)")
    print("\nDI (GT)")
    print(outputs[i]["di_text"] if outputs[i]["di_text"] else "No explicit discharge instructions")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Trained model & tokenizer loaded
✅ TEST INPUT rows: 68,589
✅ TEST OUTPUT rows: 68,589
🎯 최종 선택된 테스트 샘플 수: 3
✅ Ground-truth outputs attached for 3 samples.


------------- CASE 1/3 ---------------

[Evidence fetched: 12 blocks]
### EVIDENCE #1
disease name: Carcinoma of endocrine pancreas (disorder)
disease description: A primary malignant neoplasm of the pancreatic ISLET CELLS. Usually it involves the non-INSULIN-producing cell types, the PANCREATIC ALPHA CELLS and the pancreatic delta cells (SOMATOSTATIN-SECRETING CELLS) in GLUCAGONOMA and SOMATOSTATINOMA, respectively.
drug name: Streptozocin product
relationship attribute: may_be_treated_by

### EVIDENCE #2
disease name: Paraganglioma (disorder)
disease description: A benign or malignant neoplasm arising from paraganglia located along the sympathetic or parasympathetic nerves. Infrequently, it may arise outside the usual distribution of the sympathetic and parasympathetic paraganglia. Tumors arising from the adrenal gland medulla a

# RAG로 BHC제외하고 DI부분에만 적용

In [23]:
# =========================
# 설치 (필요시 1회)
# =========================
!pip -q install -U transformers accelerate peft trl datasets pandas bitsandbytes rich
!pip -q install langchain langchain-community faiss-cpu sentence-transformers

# =========================
# 0) 공용 import & Drive 마운트
# =========================
import os, re, json, pandas as pd, torch
from typing import List, Optional, Tuple
from collections import Counter
from google.colab import drive
drive.mount('/content/drive')

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# LangChain FAISS/E5
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# =========================
# 1) 고정 경로/파라미터
# =========================
MAX_SEQ_LENGTH = 20000
SHUFFLE_SEED   = 42
TEST_LIMIT     = 3
KEYS           = ["subject_id", "hadm_id", "note_id"]

MODEL_DIR   = "/content/drive/MyDrive/DILAB/Qwen2-7B-Instruct"
ADAPTER_DIR = "/content/drive/MyDrive/DILAB/Qwen2-7B-Instruct/adapter/original_1.3"
BASE_DATA   = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/files/note/input_output"

TEST_INPUT  = f"{BASE_DATA}/discharge_test_input.parquet"
TEST_OUTPUT = f"{BASE_DATA}/discharge_test_output.parquet"

# ==== (RAG) FAISS/E5 경로 ====
FAISS_DIR  = "/content/drive/MyDrive/DILAB/MARS/UMLS/faiss_e5_base"
E5_MODEL   = "intfloat/e5-base-v2"   # 한글 질의 섞이면 "intfloat/multilingual-e5-base"

# =========================
# 2) Qwen2 + 어댑터 로드
# =========================
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_DIR,
    quantization_config=bnb_config,
    dtype=torch.bfloat16,
    max_length=MAX_SEQ_LENGTH,
    device_map={"": 0},
    offload_buffers=True
)
trained_model = PeftModel.from_pretrained(
    base_model,
    ADAPTER_DIR,
    is_trainable=False
)
trained_model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.max_seq_length = MAX_SEQ_LENGTH

print("✅ Trained model & tokenizer loaded")

# =========================
# 3) 데이터 로드 & 샘플 선택
# =========================
test_input_df  = pd.read_parquet(TEST_INPUT)
test_output_df = pd.read_parquet(TEST_OUTPUT)
print(f"✅ TEST INPUT rows: {len(test_input_df):,}")
print(f"✅ TEST OUTPUT rows: {len(test_output_df):,}")

test_merged_df = test_input_df.merge(test_output_df, on=KEYS, how="inner")
both_empty = (test_merged_df["bhc_text"].fillna("").str.len() == 0) & \
             (test_merged_df["di_text"].fillna("").str.len() == 0)
test_merged_df = test_merged_df[~both_empty].reset_index(drop=True)

if TEST_LIMIT is not None and TEST_LIMIT < len(test_merged_df):
    sample_df = test_merged_df.sample(n=TEST_LIMIT, random_state=SHUFFLE_SEED).reset_index(drop=True)
else:
    sample_df = test_merged_df.copy()

print(f"🎯 최종 선택된 테스트 샘플 수: {len(sample_df):,}")

# =========================
# 4) 지시문(역할 설명)
# =========================
BHC_INSTR = """
You are an expert physician-writer. Write ONLY the “Brief Hospital Course (BHC)”
for THIS admission using facts explicitly present in the INPUT note.

HARD RULES
- Cite ONLY facts that appear verbatim in the input; do NOT invent meds, doses, procedures, dates.
- Prefer ≤8 bullets or a compact paragraph.
- Chronologic clarity: presentation → key findings → diagnostics → treatments/changes with reasons → response/status at discharge → pending items.
- Keep it clinical, crisp, and de-duplicated.
- DO NOT output any problem-list lines starting with '#'. If a problem list exists in the source, rewrite concisely as bullets ('-') or inline text.

OUTPUT
Brief Hospital Course
<content only; DO NOT output any other headings>
"""

DI_INSTR = """
You are an expert physician-writer. Produce “Discharge Instructions (DI)”.

EVIDENCE RULES
- You will be given "EVIDENCE BLOCKS" from a knowledge base.
- Use these evidence blocks STRICTLY to inform potential medication advice related to diagnoses.
- If an evidence block includes contraindications or 'contraindicated' terms,
  reflect this as a caution rather than recommending the drug.
- DO NOT mention diseases or drugs from the evidence unless they are also
  supported by the main "INPUT NOTE".

INPUT NOTE RULES
- Also use the "INPUT NOTE" to find patient-facing instructions.
- Include ONLY explicit instructions (medication directions/changes with dose/frequency if present;
  activity/wound care; when/where to follow up; monitoring; diet).
- If there are NO explicit instructions in the INPUT NOTE and NO relevant evidence, output exactly one line:
  No explicit discharge instructions found in source.

STYLE
- Bulleted, concise, patient-facing language.

OUTPUT
Discharge Instructions
<patient-facing bullets only; DO NOT add other sections>
"""

# =========================
# 5) (RAG) FAISS + E5 임베딩 로드 & 리트리버 (DI 전용)
# =========================
emb = HuggingFaceEmbeddings(
    model_name=E5_MODEL,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True, "batch_size": 64},
)
vs = FAISS.load_local(FAISS_DIR, emb, allow_dangerous_deserialization=True)

retriever = vs.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 8, "fetch_k": 50, "lambda_mult": 0.5}
)

def _strip_passage_prefix(text: str) -> str:
    return text.replace("passage: ", "", 1) if text.startswith("passage: ") else text

# ---- 노이즈 판정(증거/출력 공통) ----
_noise_hash_re = re.compile(r"(?:^|\s)#[A-Za-z][A-Za-z0-9\- ]{2,}")

def is_noise_block(text: str) -> bool:
    t = text.strip()
    if not t:
        return True
    if len(_noise_hash_re.findall(t)) >= 5 or t.count("#") >= 8:
        return True
    words = re.findall(r"\b[A-Za-z]{3,}\b", t)
    if len(words) > 250 and t.count(".") < 5:
        return True
    if words:
        c = Counter([w.lower() for w in words])
        topn = c.most_common(1)[0][1]
        if topn / max(1, len(words)) > 0.2:
            return True
    return False

def compress_block(b: str, max_chars=420) -> str:
    b = _strip_passage_prefix(b.strip())
    def pick(label):
        m = re.search(fr'{label}:\s*(.+)', b, re.I)
        return (label, m.group(1).strip()) if m else None
    keys = ["disease name", "disease description", "drug name", "relationship attribute"]
    lines = []
    for k in keys:
        p = pick(k)
        if p: lines.append(f"{p[0]}: {p[1]}")
    s = "\n".join(lines) if lines else b
    return s[:max_chars] + (" ..." if len(s) > max_chars else "")

def format_evidence_blocks(blocks: List[str]) -> str:
    lines = []
    for i, blk in enumerate(blocks[:8], 1):  # 최대 8개
        lines.append(f"### EVIDENCE #{i}\n{compress_block(blk)}")
    return "\n\n".join(lines)

# =========================
# RAG: 고정밀 질의 & 근거 필터
# =========================
# 1) 노트에서 췌장/질병 키워드 추출
def disease_terms_from_note(note: str) -> List[str]:
    # 주요 키워드 & 동의어 확장 가능
    terms = set()

    # 기본 앵커(장기)
    if re.search(r"\bpancrea\w*\b", note, re.I):
        terms.update(["pancreas", "pancreatic"])

    # NET/신경내분비
    if re.search(r"\b(neuroendocrine|NET\b)\b", note, re.I):
        terms.update(["neuroendocrine tumor", "islet cell tumor", "pancreatic neuroendocrine tumor"])

    # carcinoma/adenocarcinoma/ductal
    if re.search(r"\b(ductal|adenocarcinoma|carcinoma)\b", note, re.I) and re.search(r"\bpancrea\w*\b", note, re.I):
        terms.update(["pancreatic carcinoma", "pancreatic ductal adenocarcinoma"])

    # 수술/병력 기반: pancreatectomy가 있으면 췌장종양 일반 질의 추가
    if re.search(r"\bpancreatect\w*\b", note, re.I):
        terms.update(["pancreatic tumor", "neoplasm of pancreas", "pancreatic mass"])

    # 여전히 용어가 없다면 최소 anchor 질의
    if not terms and re.search(r"\bpancrea\w*\b", note, re.I):
        terms.update(["pancreatic tumor", "neoplasm of pancreas"])

    return sorted(terms)

# 2) 고정밀 RAG 질의 생성 (스니펫/플랜 전체 질의 제거)
def build_rag_queries_from_note(note: str) -> List[str]:
    terms = disease_terms_from_note(note)
    queries = []
    for t in terms:
        q = f"query: {t}"
        if q not in queries:
            queries.append(q)
    # 상한
    return queries[:6] if queries else []

# 3) 근거 블록의 질병명/설명 파싱
def _parse_evidence_meta(block: str) -> Tuple[str, str]:
    name = ""
    desc = ""
    m1 = re.search(r"(?im)^\s*disease name:\s*(.+)$", block)
    if m1:
        name = m1.group(1).strip()
    m2 = re.search(r"(?im)^\s*disease description:\s*(.+)$", block)
    if m2:
        desc = m2.group(1).strip()
    return (name, desc)

# 4) 근거 블록 필터: (a) 췌장 anchor 필수, (b) 노트 용어와 정합
def filter_evidence_blocks_by_note(note_text: str, blocks: List[str]) -> List[str]:
    if not blocks:
        return []

    note_terms = set(map(str.lower, disease_terms_from_note(note_text)))
    out = []

    for b in blocks:
        t = _strip_passage_prefix(b.strip())
        if is_noise_block(t):
            continue

        disease, desc = _parse_evidence_meta(t)
        text_all = (disease + " " + desc).lower()

        # (a) 췌장 anchor 필수
        if "pancrea" not in text_all:
            continue

        # (b) 노트 용어 정합: note_terms 중 하나라도 disease/desc에 등장
        matched = False
        for nt in note_terms:
            if nt and nt in text_all:
                matched = True
                break

        # 만약 note_terms가 빈 경우(예외적)에는 최소한 'pancrea' anchor만으로 통과
        if not note_terms:
            matched = True

        if matched:
            out.append(t)

    return out

# 5) evidence 검색: 고정밀 질의 + 필터
def retrieve_evidence(note_text: str) -> List[str]:
    queries = build_rag_queries_from_note(note_text)
    if not queries:
        return []
    docs = []
    for q in queries:
        docs.extend(retriever.get_relevant_documents(q))

    # 중복/노이즈 제거 1차
    seen = set(); uniq = []
    for d in docs:
        t = _strip_passage_prefix(d.page_content.strip())
        if t in seen or is_noise_block(t):
            continue
        seen.add(t)
        uniq.append(t)
        if len(uniq) >= 24:
            break

    # 췌장 anchor & 노트 정합 필터
    filtered = filter_evidence_blocks_by_note(note_text, uniq)
    return filtered[:12]

# (필요시) 증거에서 (질병, 약) 페어 파싱
def parse_evidence_pairs(blocks: List[str]) -> List[Tuple[str, str]]:
    pairs = []
    for b in blocks:
        if is_noise_block(b):
            continue
        dis = re.search(r"(?im)^disease name:\s*(.+)$", b)
        drg = re.search(r"(?im)^drug name:\s*(.+)$", b)
        if dis and drg:
            disease = re.split(r"[\(\[,;/]", dis.group(1).strip())[0].strip()
            drug    = re.split(r"[\(\[,;/]", drg.group(1).strip())[0].strip()
            if len(disease) >= 3 and len(drug) >= 2:
                pairs.append((disease, drug))
    return pairs

# =========================
# 6) 생성 유틸 + 노이즈 클린
# =========================
def _serialize_chat(messages: List[dict]) -> str:
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

@torch.inference_mode()
def _generate_text(model, prompt: str, max_new_tokens: int = 800) -> str:
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    gen_ids = model.generate(
        inputs.input_ids, max_new_tokens=max_new_tokens, do_sample=False,
        repetition_penalty=1.20, no_repeat_ngram_size=7,
        eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id,
    )
    gen_only = [out[len(inp):] for inp, out in zip(inputs.input_ids, gen_ids)]
    return tokenizer.batch_decode(gen_only, skip_special_tokens=True)[0].strip()

def build_bhc_prompt(note_text: str) -> str:
    messages = [
        {"role": "system", "content": BHC_INSTR},
        {"role": "user",   "content": f"INPUT NOTE:\n{note_text.strip()}"},
        {"role": "assistant", "content": ""}
    ]
    return _serialize_chat(messages)

def build_di_prompt(note_text: str, evidence_text: str) -> str:
    user_content = f"INPUT NOTE:\n{note_text.strip()}\n\n"
    if evidence_text:
        user_content += ("(For validator use ONLY. Do NOT add new items because of this.)\n"
                         f"EVIDENCE BLOCKS:\n{evidence_text}\n")
    STRICT_DI_INSTR = DI_INSTR + """
HARD CONSTRAINTS
- OUTPUT MUST BE EXTRACTIVE: every bullet must quote or paraphrase a line that exists in INPUT NOTE.
- DO NOT add any new instruction not present in INPUT NOTE.
- Evidence blocks are for internal cross-check only. If evidence conflicts with INPUT NOTE, OMIT it.
"""
    messages = [
        {"role": "system", "content": STRICT_DI_INSTR},
        {"role": "user",   "content": user_content},
        {"role": "assistant", "content": ""}
    ]
    return _serialize_chat(messages)

def assemble_final_output(bhc_text: str, di_text: str) -> str:
    out = []
    if not re.search(r'(?im)^Brief Hospital Course\s*$', bhc_text):
        out.append("Brief Hospital Course")
    out.append(bhc_text.strip())
    if di_text and di_text.strip() and di_text != "No explicit discharge instructions found in source.":
        if not re.search(r'(?im)^Discharge Instructions\s*$', di_text):
            out.append("\nDischarge Instructions")
        out.append(di_text.strip())
    elif di_text == "No explicit discharge instructions found in source.":
        out.append("\nDischarge Instructions")
        out.append(di_text.strip())
    return "\n".join(out).strip()

# ---- 원문과 연속 일치 검사 ----
def has_long_overlap(line: str, source: str, min_run: int = 8) -> bool:
    line_norm = re.sub(r'\s+', ' ', line).strip().lower()
    src_norm  = re.sub(r'\s+', ' ', source).strip().lower()
    if len(line_norm) < min_run: return False
    for i in range(0, len(line_norm) - min_run + 1):
        if line_norm[i:i+min_run] in src_norm: return True
    return False

# ---- 생성 결과 노이즈 클리너 ----
def clean_model_output(
    text: str, source_note: str, allow_hash: bool = False,
    max_hash_lines: int = 6, runaway_threshold: int = 5,
) -> str:
    lines = text.splitlines(); cleaned = []
    hash_count = runaway_hash_seq = 0
    for ln in lines:
        raw = ln.rstrip()
        if not raw.strip():
            if cleaned and cleaned[-1] == "": continue
            cleaned.append(""); continue
        if raw.lstrip().startswith("#"):
            if (not allow_hash) or (not has_long_overlap(raw, source_note, 8)):
                runaway_hash_seq += 1
                if runaway_hash_seq >= runaway_threshold: break
                continue
            if hash_count >= max_hash_lines: break
            hash_count += 1; runaway_hash_seq = 0
            cleaned.append(raw); continue
        token_words = re.findall(r"[A-Za-z]{3,}", raw)
        if (len(token_words) > 40 and raw.count(",") + raw.count(";") >= 6) and not has_long_overlap(raw, source_note, 8):
            continue
        if cleaned and cleaned[-1] == raw: continue
        runaway_hash_seq = 0; cleaned.append(raw)
    out = "\n".join(cleaned)
    out = re.sub(r"\n{3,}", "\n\n", out).strip()
    return out

# =========================
# 6-추가) DI 추출/검증/정화 유틸 (강화판)
# =========================
# 지시 동사(환자용) 패턴
_DIRECTIVE_RE = re.compile(
    r"(call|avoid|do not|don't|resume|continue|take|start|begin|initiate|increase|decrease|drink|eat|walk|ambulate|follow|lift|limit|restrict|shower|bath|swim|drive|return|contact|report|monitor|check|weigh|keep|apply|use|obtain|schedule|attend|inform|notify|seek|go to|change|elevate|rest)",
    re.I
)
# 비환자용/검사결과/영상/병리 등 제거 패턴
_NON_INSTR_RE = re.compile(
    r"(ct\s|mri\s|x-?ray|radiolog|impression|attenuation|cm\b|mm\b|final\s+report|culture|histology|pathology|specimen|biopsy|findings?:)",
    re.I
)

def extract_di_from_note(note_text: str) -> str:
    hdr_regex = re.compile(r"(?is)(^|\n)\s*(Discharge\s+Instructions?|Discharge\s+Plan|Patient\s+Instructions?)\s*:?\s*\n")
    m = hdr_regex.search(note_text)
    if not m: return ""
    start = m.end(); chunk = note_text[start:]
    nxt = re.search(r"\n[A-Z][A-Za-z ]{2,40}\s*:\s*\n", chunk)
    if nxt: chunk = chunk[:nxt.start()]
    bullets = []
    for ln in chunk.splitlines():
        s = ln.strip()
        if not s: continue
        if _NON_INSTR_RE.search(s):  # 비환자용 문장 제거
            continue
        if re.match(r"^[-*•]\s+", s) or _DIRECTIVE_RE.search(s):
            s = re.sub(r"^[-*•]\s*", "- ", s)
            bullets.append(s)
    uniq, seen = [], set()
    for b in bullets:
        if b not in seen: seen.add(b); uniq.append(b)
    return "\n".join(uniq).strip()

def normalize_bullets(text: str) -> str:
    out = []
    for ln in text.splitlines():
        s = ln.strip()
        if not s: continue
        if _NON_INSTR_RE.search(s):  # 비환자용 문장 제거
            continue
        if re.match(r"^[-*•]\s+", s):
            s = re.sub(r"^[-*•]\s*", "- ", s)
        elif re.match(r"^\d+[\.\)]\s+", s):
            s = re.sub(r"^\d+[\.\)]\s*", "- ", s)
        elif re.search(r":\s*$", s) and len(s.split()) <= 3:
            continue
        elif _DIRECTIVE_RE.search(s):
            s = "- " + s
        else:
            continue
        out.append(s)
    final, seen = [], set()
    for b in out:
        if b not in seen: seen.add(b); final.append(b)
    return "\n".join(final)

_DOSE_PAT = re.compile(r"(\b\d+(?:\.\d+)?\s*(?:mg|mcg|g|gram|ml|units?)\b|\bq\d{1,2}h\b|\bq\s?\d+\s?(?:h|hr|hrs|hours?)\b|\b(bid|tid|qid|qod|daily|once daily|twice daily)\b|\bfor\s+\d+\s+(?:day|days|weeks?)\b)", re.I)
_TRIGGER_START = re.compile(r"^\-\s*(start|begin|initiate|prescribe|commence)\b", re.I)

def _dose_fragments_in_source(bullet: str, source: str) -> bool:
    src_norm = re.sub(r"\s+", " ", source).lower()
    ok = True
    for m in _DOSE_PAT.finditer(bullet):
        frag = re.sub(r"\s+", " ", m.group(0)).strip().lower()
        if frag and frag not in src_norm: ok = False; break
    return ok

def filter_di_to_source(di_text: str, source: str, min_run: int = 12) -> str:
    keep = []
    for b in di_text.splitlines():
        s = b.strip()
        if not s or not s.startswith("- "): continue
        if re.match(r"(?i)^-+\s*discharge instructions", s): continue
        # 지시문성 필수
        if not _DIRECTIVE_RE.search(s): continue
        # 비환자용 제거
        if _NON_INSTR_RE.search(s): continue
        if not has_long_overlap(s, source, min_run=min_run): continue
        if _TRIGGER_START.match(s) and not has_long_overlap(s, source, min_run=max(min_run, 20)): continue
        if _DOSE_PAT.search(s) and not _dose_fragments_in_source(s, source): continue
        keep.append(s)
    out, seen = [], set()
    for k in keep:
        if k not in seen: seen.add(k); out.append(k)
    return "\n".join(out)

_BAD_PATTERNS = [r"stop all medications", r"wounds?\s+(must|should)\s+remain\s+open", r"Tylenol.*(Lipitor|Simvastatin|statin|Advil).*same"]
_CONFLICTS = [(r"avoid\s+(aspirin|nsaids)", r"low[- ]dose\s+aspirin")]

def sanitize_di(di_text: str) -> str:
    lines = []
    for ln in di_text.splitlines():
        if any(re.search(p, ln, re.I) for p in _BAD_PATTERNS): continue
        lines.append(ln)
    out = "\n".join(lines)
    for a, b in _CONFLICTS:
        if re.search(a, out, re.I) and re.search(b, out, re.I):
            out = re.sub(b, b + " (only if instructed by your clinician)", out, flags=re.I)
    return out.strip()

_WOUND_KW   = re.compile(r"(incision|wound|staple|steri-?strip|shower|bath|swim|dressings?)", re.I)
_ACTIVITY_KW= re.compile(r"(drive|lifting|lift|weight|activity|exercise|ambulat(e|ion))", re.I)
_EMERG_KW   = re.compile(r"(call|return|emergency|fever|chills|chest pain|shortness of breath|vomit|dehydration|black|bloody|bleeding|urine|burning)", re.I)

def ensure_core_categories(di_text: str, note_text: str) -> str:
    current_bullets = [b for b in di_text.splitlines() if b.strip()]
    seen_bullets = set(current_bullets)

    has_wound    = any(_WOUND_KW.search(b) for b in current_bullets)
    has_activity = any(_ACTIVITY_KW.search(b) for b in current_bullets)
    has_emerg    = any(_EMERG_KW.search(b) for b in current_bullets)

    need = []
    if not has_wound:    need.append(_WOUND_KW)
    if not has_activity: need.append(_ACTIVITY_KW)
    if not has_emerg:    need.append(_EMERG_KW)

    if not need: return di_text

    adds = []
    for ln in note_text.splitlines():
        s = ln.strip()
        if not s: continue
        if _NON_INSTR_RE.search(s):  # 비환자용 제외
            continue
        for pat in need:
            if pat.search(s):
                s_norm = re.sub(r"^[-*•]\s*", "- ", s)
                if not s_norm.startswith("- "): s_norm = "- " + s_norm
                if has_long_overlap(s_norm, note_text, 10) and s_norm not in seen_bullets and _DIRECTIVE_RE.search(s_norm):
                    adds.append(s_norm)
                    seen_bullets.add(s_norm)
                break

    if not adds: return di_text

    final_bullets = current_bullets + adds
    final_text = normalize_bullets("\n".join(final_bullets))
    final_text = sanitize_di(final_text)
    return final_text

# =========================
# (추가) 정답(GT) 리스트
# =========================
outputs = sample_df[["bhc_text", "di_text"]].fillna("").to_dict("records")
print(f"✅ Ground-truth outputs attached for {len(outputs)} samples.")

# =========================
# 7) 실행: BHC(원문), DI(추출 우선 → 실패 시 LLM, 필터/보강)
# =========================
N = min(3, len(sample_df))
for i in range(N):
    note_text = sample_df.loc[i, "text"]
    print(f"\n\n------------- CASE {i+1}/{N} ---------------")

    # 1) BHC: 원문만 사용 (RAG 미사용)
    bhc_prompt = build_bhc_prompt(note_text)
    bhc_raw    = _generate_text(trained_model, bhc_prompt, max_new_tokens=800)
    bhc_text   = clean_model_output(bhc_raw, note_text, allow_hash=False)

    # 2) RAG 증거 검색 (검증용): 고정밀 쿼리 + 필터
    retrieved_blocks = retrieve_evidence(note_text)
    evidence_text = format_evidence_blocks(retrieved_blocks)
    if evidence_text:
        print(f"\n[Evidence fetched (filtered): {len(retrieved_blocks)} blocks]\n{evidence_text}")
    else:
        print("\n[No relevant evidence fetched]")

    # 3) DI: (A) 추출형 시도
    di_text = extract_di_from_note(note_text)
    source_of_di = "Extracted" if di_text and len(di_text.splitlines()) >= 2 else "None"

    # 3-1) 추출 실패 시 (B) LLM 생성 시도
    if source_of_di == "None":
        di_prompt = build_di_prompt(note_text, evidence_text)
        di_raw    = _generate_text(trained_model, di_prompt, max_new_tokens=600)
        di_text   = clean_model_output(di_raw, note_text, allow_hash=False)
        source_of_di = "LLM_Generated"

    # 4) 불릿 정규화
    di_text = normalize_bullets(di_text)

    # 5) LLM 생성된 경우에만 출처-일치/용량/지시문 필터 엄격 적용
    if source_of_di == "LLM_Generated":
        di_text = filter_di_to_source(di_text, note_text, min_run=12)

    # 6) 위험/오정보 정화 및 모순 중화
    di_text = sanitize_di(di_text)

    # 7) 핵심 카테고리 보장 (부족분은 원문 추출로 보강)
    di_text = ensure_core_categories(di_text, note_text)

    # 8) 비어 있으면 규칙에 따라 한 줄 출력
    if not di_text.strip():
        di_text = "No explicit discharge instructions found in source."

    # 9) 합치기
    final_text = assemble_final_output(bhc_text, di_text)
    print("\n\n-------------------FINAL MODEL OUTPUT-------------------")
    print(final_text)

    # 10) GT(정답)도 함께 출력
    print(f"\n\n-------------------output (GT)------------------------")
    print("BHC (GT)")
    print(outputs[i]["bhc_text"] if outputs[i]["bhc_text"] else "(empty)")
    print("\nDI (GT)")
    print(outputs[i]["di_text"] if outputs[i]["di_text"] else "No explicit discharge instructions")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Trained model & tokenizer loaded
✅ TEST INPUT rows: 68,589
✅ TEST OUTPUT rows: 68,589
🎯 최종 선택된 테스트 샘플 수: 3
✅ Ground-truth outputs attached for 3 samples.


------------- CASE 1/3 ---------------

[Evidence fetched (filtered): 11 blocks]
### EVIDENCE #1
disease name: Carcinoma of endocrine pancreas (disorder)
disease description: A primary malignant neoplasm of the pancreatic ISLET CELLS. Usually it involves the non-INSULIN-producing cell types, the PANCREATIC ALPHA CELLS and the pancreatic delta cells (SOMATOSTATIN-SECRETING CELLS) in GLUCAGONOMA and SOMATOSTATINOMA, respectively.
drug name: Streptozocin product
relationship attribute: may_be_treated_by

### EVIDENCE #2
disease name: Neoplasm of pancreas (disorder)
disease description: Tumors or cancer of the PANCREAS. Depending on the types of ISLET CELLS present in the tumors, various hormones can be secreted: GLUCAGON from PANCREATIC ALPHA CELLS; INSULIN from PANCREATIC BETA CELLS; and SOMATOSTATIN from the SOMATOSTATIN-SECRETING 

# RAG없는 버전

In [6]:
# ====== 0. 필요 모델 import & 다운로드 ======

!pip install -U transformers accelerate peft trl datasets pandas bitsandbytes rich

import os, re, pandas as pd, torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from typing import List
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')


# ====== 1. 필요 변수 선언 ======
MAX_SEQ_LENGTH = 20000        # 최대 시퀀스 토큰 길이
SHUFFLE_SEED = 42         # 랜덤 시드
TEST_LIMIT = 3    # 테스트할 데이터 개수
KEYS = ["subject_id", "hadm_id", "note_id"] # 각 데이터셋을 불러올 기준은 KEY 목록


MODEL_DIR = "/content/drive/MyDrive/DILAB/Qwen2-7B-Instruct"
ADAPTER_DIR = "/content/drive/MyDrive/DILAB/Qwen2-7B-Instruct/adapter/original_1.3"
BASE_DATA   = "/content/drive/MyDrive/DILAB/MARS/mimic-iv-note_2.2/files/note/input_output"

TEST_INPUT  = f"{BASE_DATA}/discharge_test_input.parquet"
TEST_OUTPUT = f"{BASE_DATA}/discharge_test_output.parquet"
TRAIN_INPUT = f"{BASE_DATA}/discharge_train_input.parquet"
TRAIN_OUTPUT= f"{BASE_DATA}/discharge_train_output.parquet"


# ====== 2. 모델 로드하기 ======
# --- 2-1. 4비트 양자화 ---
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# --- 2-2. original_model & trained_model 모델 로드 ---
original_model = AutoModelForCausalLM.from_pretrained(
    MODEL_DIR,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    max_length=MAX_SEQ_LENGTH,
    device_map={"": 0},
    offload_buffers=True
)

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_DIR,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    max_length=MAX_SEQ_LENGTH,
    device_map={"": 0},
    offload_buffers=True
)

trained_model = PeftModel.from_pretrained(
    base_model,
    ADAPTER_DIR,
    is_trainable=False # 추론용이면 False
)

# --- 2-3. 모델 평가모드 전환 ---
trained_model.eval()

# ---- 2-4. 토크나이저 로드 ---
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, trust_remote_code=True)
  # -> trust_remote_code : 모델 저장소에 커스텀 토크나이저 코드가 있으면, 신뢰하고 불러오겠다는 의미
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
  # -> 패딩을 오른쪽에 붙이도록 설정
  # 토큰화 후 길이가 다른 문장들을 batch로 묶을 때, 짧은 문장의 뒤쪽에 [PAD] 토큰을 붙인다.
tokenizer.max_seq_length = MAX_SEQ_LENGTH
  # -> 최대 시퀀스 길이 설정


print("\n✅ Base Model & Trained Model & 토크나이저 로드 완료!")


# ====== 3. 랜덤으로 TEST 데이터 뽑기 ======
# --- 3-1. 데이터 로드하기 ---
test_input_df = pd.read_parquet(TEST_INPUT)
test_output_df = pd.read_parquet(TEST_OUTPUT)

print(f"✅ TEST INPUT rows: {len(test_input_df):,}") # :, 포맷 지정자=> 숫자를 천 단위 구분 쉼표로 표시함
print(f"✅ TEST OUTPUT rows: {len(test_input_df):,}")

# --- 3-2. KEYS 기준으로 input, output 파일을 inner join (두 테이블의 컬럼들이 합쳐짐) ---
test_merged_df = test_input_df.merge(test_output_df, on=KEYS, how="inner")
  # -> on: 어떤 KEY들을 기준으로 join할 것인지
  # -> how: 어떤 방식으로 join할 것인지
print(f"✅ TEST MERGED rows: {len(test_merged_df):,}")

# --- 3-3. 병합한 데이터들 중, BHC나 DI가 비어있다면 해당 데이터는 제거함 ---
both_empty = (test_merged_df["bhc_text"].fillna("").str.len() == 0) & \
             (test_merged_df["di_text"].fillna("").str.len() == 0)
  # -> fillna(""): NaN 값을 빈 문자열로 대체하는 코드
  # -> .str.len(): 문자열로 변환한 뒤 해당 문자열의 길이를 반환
test_merged_df = test_merged_df[~both_empty].reset_index(drop=True)
  # -> ~both_empty: both_empty에는 각 행마다 True/False가 있는데, "~" 비트 Not 연산자로 값을 뒤집는다
  # -> reset_index(drop=True) : 데이터 프레임의 인덱스를 0부터 순서대로 다시 매겨주는 함수
    # drop=True 옵션을 주면, 기존 인덱스를 새로운 컬럼으로 남기지 않고 완전히 버린다.

# --- 3-4. 필터링된 데이터셋에서 TEST_LIMIT만큼 랜덤시드로 데이터 꺼내오기 ---
# TEST_LIMIT가 None이 아니고, 전체 데이터보다 작을 때만 꺼내오기
if TEST_LIMIT is not None and TEST_LIMIT < len(test_merged_df):
    sample_df = test_merged_df.sample(n=TEST_LIMIT, random_state=SHUFFLE_SEED).reset_index(drop=True)
    # .sample(): 난수 생성기를 사용해서 데이터프레임의 행 중 일부를 무작위로 선택함.
    # reset_index(drop=True) : 무작위로 섞이면서 기존 인덱스가 섞였기에, 0부터 순서대로 인덱스를 다시 매겨준다.
else:
  # TEST_LIMIT보다 전체 데이터가 작은 경우에는 해당 데이터 copy해서 그대로 사용하기
    sample_df = train_merged_df.copy()

print(f"🎯 최종 선택된 훈련용 샘플 수: {len(sample_df):,}")


# ====== 4. 프롬프트 설정하기 ======
# --- 4-1. 지시문 설정 ---
INSTRUCTION = (
    """
You are an expert physician-writer who crafts hospital discharge summaries.

OBJECTIVE
- Produce the best possible “Brief Hospital Course” (BHC) and “Discharge Instructions” (DI) for this admission.
- Be accurate, concise, and high-yield.

EVIDENCE RULES
- Use ONLY facts explicitly present in the input for this admission.
- If something is not stated, omit it (do not guess or add placeholders).
- Preserve all numbers/units/doses/medication names exactly as written when cited.
- Expand abbreviations on first mention (e.g., “glomerular basement membrane (GBM)”).

STYLE
- English, clinical, crisp, and readable.
- Prefer bullets for scannability; short paragraphs are acceptable.
- De-duplicate; avoid narrative fluff.
- Sensible ordering: presentation → key findings → diagnostic workup → treatments/changes (with stated rationale) → clinical response/status at discharge → pending studies/follow-up items.

OUTPUT FORMAT (exactly two top-level headings; no others)
Brief Hospital Course
- Summarize the course of this admission only.
- Include: presenting problem; salient exam/imaging/labs (with values only if meaningful); pivotal decisions and treatments (med changes with stated reasons); clinical response/trends; status at discharge; explicitly listed pending items to follow up.

Discharge Instructions
- Patient-facing, concise, and strictly derived from input directions.
- Include only actionable items explicitly present in the input, such as:
  • medication directions or changes (dose/formulation/frequency as written),
  • ordered labs/procedures and when to obtain them (if specified),
  • monitoring tasks (e.g., glucose logs) and diet/activity instructions if stated,
  • whom/where to follow up if named in the input.
- Do NOT restate the entire medication list unless it is explicitly provided as patient instructions.
- Do NOT add generic advice (e.g., hydration, exercise, smoking cessation) unless the input states it.

GUARDRAILS
- No new clinicians, appointments, or tests beyond the input.
- Do not invent demographics/PHI (age, phone, MRN, addresses).
- Do not label values as high/low/improved unless the input does so (e.g., *, H, L, or explicit wording).

LENGTH TARGETS (guidance, not strict)
- BHC: ≤ 8 bullets or a short paragraph of comparable density.
- DI: 4–10 bullets or a short paragraph.

FINALIZATION (silent self-check before output; do not print the checklist)
- Exactly two headings are present and correctly titled.
- Every sentence is supported by the input.
- All doses/units/numbers match the input verbatim when cited.
- BHC covers presentation, key findings, treatments/changes with stated reasons, response/status, and pending items (if any).
- DI contains only actionable patient-facing directions explicitly provided in the input.
    """
)

# --- 4-2. TEST용 데이터들 & 지시문을 Chat 형식으로 프롬프트 변환 ---
test_prompts = []
outputs = []
print("🪄 Chat-format로 변환 중...")

# 데이터 프레임의 iterrows() 함수로 for문 순회하여, 각 row 꺼내오기
for _, row in tqdm(sample_df.iterrows(), total=len(sample_df), desc="Formatting", ncols=100):
            # ㄴ> tqdm이 총 길이를 알고 있어서 정확한 % 표시 가능
            # ㄴ> 설명, 출력 길이 조절
  # system 메시지 (지시문)
  system_msg = {"role": "system", "content": INSTRUCTION}

  # user: input 텍스트(학습 데이터)
  user_msg = {"role": "user", "content": row["text"].strip()}

  # assistant: 모델이 출력할 데이터
  assistant_msg = {"role": "assistant", "content":""}

  # messages 구조로 위 3가지 묶은 후 test_prompts 리스트에 추가하기
  test_prompts.append([
      system_msg,
      user_msg,
      assistant_msg
  ])

  # output 결과 추가
  output = {"bhc" : row["bhc_text"], "di" : row["di_text"]}
  outputs.append(output)
print(f"✅ Chat-format 변환 완료! 총 {len(test_prompts):,}개 샘플")

# <<예시>>
# test_prompts = [
#   [
#     {"role": "system", "content": "You are a medical summarization model..."},
#     {"role": "user", "content": "환자는 흉통으로 내원하여 ..."},
#     {"role": "assistant", "content": ""}
#   ],
#   [
#     {"role": "system", "content": "You are a medical summarization model..."},
#     {"role": "user", "content": "입원 중 발열로 검사 시행 ..."},
#     {"role": "assistant", "content": ""}
#   ]
# ]

# --- 4-3. TEST용 chat 형식 프롬프트 apply_chat_template()으로 하나의 문자열로 직렬화 ---
# 직렬화한 text를 Dataset의 "text" 필드에 넣는다.
print("\n🪄 chat 형식 프롬프트 -> 문자열로 직렬화 시작")
prompts = [
    tokenizer.apply_chat_template(
        prompt,
        tokenize=False,             # 변환된 텍스트를 토큰화하지 않고 문자열 그대로 반환 (True로 하면 토큰 id 리스트를 반환함)
        add_generation_prompt=True # 마지막에 "Assistant:" 같은 답변 시작 프롬프트 붙일지 여부 지정
) for prompt in test_prompts
]
  # -> 토크나이저에 정의된 chat template을 적용해서, 모델이 이해할 수 있는 입력 문자열로 변환됨

print("✅ chat 형식 프롬프트 -> 문자열로 직렬화 완료!")
print("📁 직렬화된 데모 텍스트 문자열:")
print(prompts[0])


# ====== 5. 한개의 prompt에 대한 모델 결과 출력하는 함수 생성 ======
def generate_model_output(model, prompt):
    # --- 5-1. prompt 토크나이징 하기 ---
    model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda") # 파이썬 텐서 방식으로 return한다는 속성
    # --- 5-2. 모델에 입력 토큰 넣기 ---
    generated_ids = model.generate(
        model_inputs.input_ids,
        # temperature=0.0,
        # top_p=1.0,
        max_new_tokens=MAX_SEQ_LENGTH,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )

    # --- 5-3. 새로 생성된 부분만 떼어내기 ---
    # 각 배치마다 output_ids에는 [입력+생성] 구간 전체가 들어있음
    # input_ids의 길이만큼 앞부분(입력 부분)을 잘라내고, 뒤쪽 생성 부분만 남기는 코드임
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    # --- 5-4. 토큰 ID -> 텍스트로 복원 ---
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
      # -> batch_decode() : 여러 개의 토큰 시퀀스를 한 번에 문자열로 복원함
      # -> skip_special_tokens=True : 특수 토큰 제거
      # -> [0] : 보통 한개의 응답만 생성하므로, 배치에서 첫 번째 샘플만 꺼낸다.
    return response

# ====== 6. 각 프롬프트별 input & original_model & trained_model & output 정답 출력 결과 보기 ======
i=0
for prompt in prompts:
  print(f"\n\n-------------{i+1}번째 예시 ---------------")
  original_response = generate_model_output(original_model, prompt)
  trained_response = generate_model_output(trained_model, prompt)

  print("\n\n✅input:")
  print(prompt)

  print("\n\n✅original_model 출력 결과:")
  print(original_response)

  print("\n\n✅trained_model 출력 결과:")
  print(trained_response)

  print(("\n\n 📌output:"))
  print("\t📁bhc:")
  print(outputs[i]["bhc"])
  print("\t📁di:")
  print(outputs[i]["di"])
  i += 1




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]


✅ Base Model & Trained Model & 토크나이저 로드 완료!
✅ TEST INPUT rows: 68,589
✅ TEST OUTPUT rows: 68,589
✅ TEST MERGED rows: 68,589
🎯 최종 선택된 훈련용 샘플 수: 3
🪄 Chat-format로 변환 중...


Formatting: 100%|███████████████████████████████████████████████████| 3/3 [00:00<00:00, 3338.53it/s]

✅ Chat-format 변환 완료! 총 3개 샘플

🪄 chat 형식 프롬프트 -> 문자열로 직렬화 시작
✅ chat 형식 프롬프트 -> 문자열로 직렬화 완료!
📁 직렬화된 데모 텍스트 문자열:
<|im_start|>system

You are an expert physician-writer who crafts hospital discharge summaries.

OBJECTIVE
- Produce the best possible “Brief Hospital Course” (BHC) and “Discharge Instructions” (DI) for this admission.
- Be accurate, concise, and high-yield.

EVIDENCE RULES
- Use ONLY facts explicitly present in the input for this admission.
- If something is not stated, omit it (do not guess or add placeholders).
- Preserve all numbers/units/doses/medication names exactly as written when cited.
- Expand abbreviations on first mention (e.g., “glomerular basement membrane (GBM)”).

STYLE
- English, clinical, crisp, and readable.
- Prefer bullets for scannability; short paragraphs are acceptable.
- De-duplicate; avoid narrative fluff.
- Sensible ordering: presentation → key findings → diagnostic workup → treatments/changes (with stated rationale) → clinical response/status at dis



✅input:
<|im_start|>system

You are an expert physician-writer who crafts hospital discharge summaries.

OBJECTIVE
- Produce the best possible “Brief Hospital Course” (BHC) and “Discharge Instructions” (DI) for this admission.
- Be accurate, concise, and high-yield.

EVIDENCE RULES
- Use ONLY facts explicitly present in the input for this admission.
- If something is not stated, omit it (do not guess or add placeholders).
- Preserve all numbers/units/doses/medication names exactly as written when cited.
- Expand abbreviations on first mention (e.g., “glomerular basement membrane (GBM)”).

STYLE
- English, clinical, crisp, and readable.
- Prefer bullets for scannability; short paragraphs are acceptable.
- De-duplicate; avoid narrative fluff.
- Sensible ordering: presentation → key findings → diagnostic workup → treatments/changes (with stated rationale) → clinical response/status at discharge → pending studies/follow-up items.

OUTPUT FORMAT (exactly two top-level headings; no others)